In [62]:
import pandas as pd 

In [63]:
df=pd.read_csv("spotify_millsongdata.csv")


In [64]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [65]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [66]:
df.shape


(57650, 4)

In [67]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [68]:
df=df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [69]:
df.head(10)

,artist,song,text
0,Britney Spears,Lace And Leather,"French fingertips, \r\nRed lips \r\nBitch is..."
1,Weezer,Troublemaker,Put me in \r\nA special school \r\n'Cause I ...
2,John Waite,When I See You Smile,Sometimes I wonder \r\nIf I'd ever make it th...
3,Johnny Cash,25 Minutes To Go,Well they're building a gallows outside my cel...
4,Divine,Missing U,I ain't missing you \r\nDon't care what you d...
5,Deep Purple,One More Rainy Day,"Changes, visible changes \r\nSky looking so f..."
6,Emmylou Harris,Coat Of Many Colors,(Dolly Parton) \r\n \r\nBack through the yea...
7,Helloween,Hocus Pocus,Yodeadodoyodeadodoyodeadodoyodeadodo \r\nYode...
8,Carly Simon,Davy,"Davy, is there a song out there in the night ..."
9,Clash,Garageland,Back in the garage with my bullshit detector ...


In [70]:
df['text'][0]

"French fingertips,  \r\nRed lips  \r\nBitch is dangerous  \r\nCotton candy kiss  \r\nCan't wait for my sugar rush  \r\n  \r\nCan't take it no more  \r\nI got to have more tonight  \r\nThis feeling's so strong  \r\nI'm putting you on tonight  \r\n  \r\nAlright, let's go!  \r\n  \r\n[Chorus]  \r\nBaby take a seat  \r\nEyes on me this is my show  \r\nYour one and only pleasure  \r\nAll decked in lace and leather  \r\nFantasy, courtesy of me  \r\nBaby let's go  \r\nWatch me apply the pressure  \r\nAll decked in lace and leather  \r\n  \r\nHeels 6 inch  \r\nMakes a boy wanna bite his lip  \r\nLook but don't touch  \r\nUnless you want to lose your innocence  \r\n  \r\nCan't take it no more  \r\nI got to have more tonight  \r\nThis feelings so strong  \r\nI'm putting you on tonight  \r\n  \r\n[Chorus: x2]\r\n\r\n"

In [71]:
df.shape

(5000, 3)

Text cleaning

In [72]:
df['text']=df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ',regex = True)

In [73]:
df.tail(5)

,artist,song,text
4995,Europe,Hero,it's not like i wouldn't dream \r if you had ...
4996,David Allan Coe,Sweet Angeline,sweet angeline i hope nobody ever hurts you an...
4997,U2,Electrical Storm,"the sea, it swells like a sore head \r and th..."
4998,Dolly Parton,I Don't Believe You've Met My Baby,"last night, my tears they were fallin' \r i w..."
4999,New Order,Ultraviolence,who saw those dark eyes \r who saw those dark...


In [74]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)

In [75]:
token("You are beautiful, beauty!")


'you are beauti , beauti !'

In [76]:
df['text'] = df["text"].apply(lambda x: tokenization(x))

In [77]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [78]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [79]:
similarity[0]

array([1.        , 0.04263393, 0.08583241, ..., 0.04286687, 0.04118323,
       0.02772887])

In [82]:
df[df['song'] == 'Ultraviolence'].index[0]

np.int64(4999)

In [85]:
def recommender(song_name):
    idx = df[df['song']==song_name].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True, key = lambda x:x[1])
    songs = []
    for s_id in distances[1:21]:
        songs.append(df.iloc[s_id[0]].song)
    return songs

In [86]:
recommender('Ultraviolence')

['2 Years On',
 "Daddy's Song",
 'A Year',
 'Wasted Years',
 'After All The Good Is Gone',
 'This Must Be The Place I Waited Years To Leave',
 'All Through The Years',
 'Take Me For What I Am',
 'Please Come Home',
 'Deep',
 'Million Years Ago',
 'Caught In The Balance',
 'Loving You Again',
 'The First Time Ever I Saw Your Face',
 'Man Of The Year',
 'Gone, Gone Blues',
 'I Witness',
 'When Things Explode',
 'Still Crazy After All These Years',
 'Heresy']

In [87]:
import pickle

In [88]:
pickle.dump(similarity, open("similar", "wb"))

In [89]:
pickle.dump(df, open("df", "wb"))